In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np

In [3]:
imdb_reviews = pd.read_csv("imdb_reviews.csv")
test_reviews = pd.read_csv("test_reviews.csv")

In [4]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [5]:
# this file contain mapping of all the words to corresponding integers
word_index = pd.read_csv("word_indexes.csv")

In [6]:
word_index.head(n=10)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [7]:
# converting dataframe to dictionary
# words as keys and integers as values
word_index = dict(zip(word_index.Words,word_index.Indexes))

In [19]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [20]:
# encoding reviews as integers
def review_encoder(text):
    arr=[word_index[word] for word in text]
    return arr

In [21]:
# splitting the datasets
train_data,train_labels = imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data,test_labels = test_reviews['Reviews'],test_reviews['Sentiment']

In [22]:
# split the strings on the bases of spaces
train_data=train_data.apply(lambda review : review.split())
test_data=test_data.apply(lambda review : review.split())

In [23]:
train_data[0]

['<START',
 'this',
 'film',
 'was',
 'just',
 'brilliant',
 'casting',
 'location',
 'scenery',
 'story',
 'direction',
 "everyone's",
 'really',
 'suited',
 'the',
 'part',
 'they',
 'played',
 'and',
 'you',
 'could',
 'just',
 'imagine',
 'being',
 'there',
 'robert',
 '<UNK>',
 'is',
 'an',
 'amazing',
 'actor',
 'and',
 'now',
 'the',
 'same',
 'being',
 'director',
 '<UNK>',
 'father',
 'came',
 'from',
 'the',
 'same',
 'scottish',
 'island',
 'as',
 'myself',
 'so',
 'i',
 'loved',
 'the',
 'fact',
 'there',
 'was',
 'a',
 'real',
 'connection',
 'with',
 'this',
 'film',
 'the',
 'witty',
 'remarks',
 'throughout',
 'the',
 'film',
 'were',
 'great',
 'it',
 'was',
 'just',
 'brilliant',
 'so',
 'much',
 'that',
 'i',
 'bought',
 'the',
 'film',
 'as',
 'soon',
 'as',
 'it',
 'was',
 'released',
 'for',
 '<UNK>',
 'and',
 'would',
 'recommend',
 'it',
 'to',
 'everyone',
 'to',
 'watch',
 'and',
 'the',
 'fly',
 'fishing',
 'was',
 'amazing',
 'really',
 'cried',
 'at',
 'the

In [24]:
# encoding this data as integers
train_data = train_data.apply(review_encoder)
test_data = test_data.apply(review_encoder)

In [25]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [26]:
def encode_sentiments(sentiment):
    if sentiment=='positive':
        return 1
    else:
        return 0

train_labels = train_labels.apply(encode_sentiments)
test_labels = test_labels.apply(encode_sentiments)

In [27]:
# padding to make all reviews of equal size
train_data = keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [28]:
# embedding will convert each word into length of 16
# and each review into length of 500
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                       keras.layers.GlobalAveragePooling1D(),
                       keras.layers.Dense(16,activation='relu'),
                       keras.layers.Dense(1,activation='sigmoid')])

In [29]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 7s 55ms/step - loss: 0.6915 - accuracy: 0.5502 - val_loss: 0.6891 - val_accuracy: 0.6560
Epoch 2/30
49/49 [==============================] - 2s 47ms/step - loss: 0.6828 - accuracy: 0.6959 - val_loss: 0.6757 - val_accuracy: 0.7170
Epoch 3/30
49/49 [==============================] - 2s 48ms/step - loss: 0.6599 - accuracy: 0.7346 - val_loss: 0.6464 - val_accuracy: 0.7489
Epoch 4/30
49/49 [==============================] - 3s 64ms/step - loss: 0.6189 - accuracy: 0.7742 - val_loss: 0.6012 - val_accuracy: 0.7881
Epoch 5/30
49/49 [==============================] - 3s 60ms/step - loss: 0.5647 - accuracy: 0.8113 - val_loss: 0.5488 - val_accuracy: 0.8082
Epoch 6/30
49/49 [==============================] - 3s 52ms/step - loss: 0.5047 - accuracy: 0.8366 - val_loss: 0.4948 - val_accuracy: 0.8292
Epoch 7/30
49/49 [==============================] - 3s 66ms/step - loss: 0.4488 - accuracy: 0.8546 - val_loss: 0.4486 - val_accuracy: 0.8436
Epoch 8/30
49

In [31]:
loss,accuracy = model.evaluate(test_data,test_labels)

782/782 [==============================] - 1s 2ms/step - loss: 0.2909 - accuracy: 0.8851


In [32]:
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [33]:
idx = np.random.randint(1,1000)

In [34]:
user_review = test_reviews.loc[idx]
print(user_review)

Reviews      <START this movie was a failure as a comedy an...
Sentiment                                             negative
Name: 431, dtype: object


In [35]:
user_review = test_data[idx]
user_review = np.array([user_review])
if(model.predict(user_review) > 0.5).astype("int32"):
    print("positive sentiment")
else:
    print("negative sentiment")

negative sentiment
